In [1]:
import os,json
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

# ACE05

In [31]:
path_folder='/home/xhsun/NLP/NER/NerData/ace2005'
files_path=os.listdir(path_folder)
print(files_path)

['mrc-ner.dev', 'mrc-ner.test', 'mrc-ner.train']


In [4]:
with open("/home/xhsun/NLP/NER/NerData/ace2005/mrc-ner.train") as f:
    data=json.load(f)

In [6]:
data[0]

{'context': "BEGALA Well , we ' ll debate that later on in the show .",
 'end_position': [],
 'entity_label': 'GPE',
 'impossible': True,
 'qas_id': '0.1',
 'query': 'geographical political entities are geographical regions defined by political and or social groups such as countries, nations, regions, cities, states, government and its people.',
 'span_position': [],
 'start_position': []}

In [40]:
def process_examples(examples):
    '''
    传进来的examples都是一个question对应的example
    '''
    sentence=examples[0]['context']
    sentence_list=sentence.split()
    entlabel_list=['O']*len(sentence_list)
    for example in examples:
        assert example['context']==sentence
        span_positions=example['span_position']
        impossible=str(example['impossible']).lower()
        if impossible=="true":
            assert span_positions==[]
            continue
        assert span_positions!=[]
        entity_label=example['entity_label']

        for start_end in span_positions:
            start,end=list(map(int,start_end.split(';')))
            #采用BMES标记
            if start==end:
                if entlabel_list[start]=='O':
                    entlabel_list[start]=('S-'+entity_label)
                else:
                    #嵌套实体
                    if entity_label not in entlabel_list[start]:
                        #如果之前已经标记为M-ORG，那么就不再标记为S-ORG了，但是
                        #如果之前标记的是M-PER，那么仍然会标记为S-ORG
                        entlabel_list[start]+=('+S-'+entity_label)
            else:
                for pos in range(start,end+1):
                    if pos==start:
                        if entlabel_list[pos]=='O':
                            entlabel_list[pos]='B-'+entity_label
                        else:
                            #嵌套实体
                            if entity_label not in entlabel_list[start]:
                                entlabel_list[pos]+=('+B-'+entity_label)
                            else:
                                #如果发现实体已经被打上同样的tag，那么就不再标记
                                break
                    elif pos<end:
                        if entlabel_list[pos]=='O':
                            entlabel_list[pos]='M-'+entity_label
                        else:
                            #嵌套实体
                            entlabel_list[pos]+=('+M-'+entity_label)                        
                    else:
                        assert pos==end
                        if entlabel_list[pos]=='O':
                            entlabel_list[pos]='E-'+entity_label
                        else:
                            #嵌套实体
                            entlabel_list[pos]+=('+E-'+entity_label)
        
    return sentence_list,entlabel_list

def process_ace_data(path_folder,write_path):
    i=0
    files_path=os.listdir(path_folder)
    print(files_path)
    for file_name in files_path:
        with open(os.path.join(path_folder,file_name)) as f:
            data=json.load(f)
        sentence_count=0
        mode=file_name.split('.')[-1]
        f_write=open(os.path.join(write_path,mode),'w')
        while i<len(data)-2:
            example=data[i]
            examples_for_i=[example]
            qas_id=int(example['qas_id'].split('.')[0])
            j=i+1
            while j<len(data):
                if int(data[j]['qas_id'].split('.')[0])==qas_id:
                    examples_for_i.append(data[j])
                else:
                    break
                j+=1
            i=j
            sentence_list_i,entlabel_list_i=process_examples(examples=examples_for_i)
            assert len(sentence_list_i)==len(entlabel_list_i)
            for token,entlabel in zip(sentence_list_i,entlabel_list_i):
                f_write.write(token+'\t'+entlabel+'\n')
            f_write.write('\n')
            sentence_count+=1
        print(file_name,sentence_count)

In [41]:
process_ace_data(path_folder=path_folder,
                 write_path='/home/xhsun/Desktop/gitRepositories/DP-Relatedwork-BrieflySummarize/data/ACE05/')

['mrc-ner.dev', 'mrc-ner.test', 'mrc-ner.train']
mrc-ner.dev 971
mrc-ner.test 89
mrc-ner.train 6239


In [7]:
sentence="We ' ll have a couple of experts come out , so I ' ll withhold my comments until then ."

In [10]:
sentence.split()[4:7+1]

['a', 'couple', 'of', 'experts']

In [11]:
sentence.split()[12:12+1]

['I']

In [12]:
sentence.split()[16:16+1]

['my']

In [15]:
str(False)

'False'

In [21]:
start_end='89;34'
list(map(int,start_end.split(';')))

[89, 34]